In [70]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
import numpy as np
import os
import regex
# import fitz
# import cv2

In [72]:
GLOBAL_INPUT_DIR = '../data/input/'
GLOBAL_OCR = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

In [79]:
input_path = os.path.join(GLOBAL_INPUT_DIR, os.listdir(GLOBAL_INPUT_DIR)[0])

def churn(input_file):
    # Reject input file if it is not a PDF
    if os.path.splitext(input_file)[-1] != '.pdf':
        raise Exception('Input file is not a PDF.')
    
    # OCR the entire PDF
    result = GLOBAL_OCR.ocr(input_file, cls=True)

    # Consider the first page of the PDF to classify the general type of document.
    # The general classification will be used to do a type-specific analysis which will give the desired output.
    first_page_texts = [line[1][0] for line in result[0]]
    merged_text = ' '.join(first_page_texts).lower()
    category_counts = {}
    categories = {
        'nlvl' : r'normal\s*\(\w+\)\s*level', # for n-level
        'olvl' : r'ordinary\s*level', # for o-level
        'ite' : r'national\s*ite\s*certificate\sin', # for NITEC or Higher NTIEC
        'poly' : r'polytechnic', # for poly
        'ted' : r'technical\s*engineer\s*diploma', # for TED
        'alvl' : r'advanced\s*level', # for a-level
        'ib' : r'baccalaureate', # for IB
        'nush' : r'nus\s*high' # for NUS High
    }
    for category in categories:
        pattern = '(' + categories[category] + '){e<=1}'
        category_counts[category] = len(regex.findall(pattern, merged_text))
    max_count_category = max(category_counts, key=category_counts.get)
    if max_count_category == 'psle':
        print('psle')
    elif max_count_category == 'nlvl':
        print('nlvl')
    elif max_count_category == 'olvl':
        print('olvl')
    elif max_count_category == 'ite':
        # Further differentiate between NITEC and H.NITEC
        if len(regex.findall(r'(higher\s*national\s*ite\s*certificate\sin){e<=1}', merged_text)):
            # its a Higher NITEC
            print('higher_nitec')
        else:
            print('nitec')
    elif max_count_category == 'poly':
        # Further differentiate between the 5 local polys
        polys = {
            'rp': r'republic',
            'np': r'ngee\s*ann',
            'nyp': r'nanyang',
            'tp': r'temasek',
            'sp': r'singapore'
        }
        poly_counts = {}
        for poly in polys:
            poly_pattern = '(' + polys[poly] + '){e<=1}'
            poly_counts[poly] = len(regex.findall(poly_pattern, merged_text))
        max_count_poly = max(poly_counts, key=poly_counts.get)
        if max_count_poly == 'rp':
            print('rp')
        elif max_count_poly == 'np':
            print('np')
        elif max_count_poly == 'nyp':
            print('nyp')
        elif max_count_poly == 'tp':
            print('tp')
        elif max_count_poly == 'sp':
            print('sp')
    elif max_count_category == 'ted':
        print('ted')
    elif max_count_category == 'alvl':
        print('alvl')
    elif max_count_category == 'ib':
        print('ib')
    elif max_count_category == 'nush':
        print('nush')

churn(input_path)


olvl
